In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from hyperopt import hp, fmin, tpe, Trials
import itertools

In [2]:
train = '/mnt/NewVolume/DUMP/Notebook/xray_analytics_cnn/data/BoneFractureDataset/training'
test = '/mnt/NewVolume/DUMP/Notebook/xray_analytics_cnn/data/BoneFractureDataset/testing'

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
#Preprocessing

In [4]:
def scale_images(x, y):
    return x / 255, y

In [5]:
train_data = tf.keras.utils.image_dataset_from_directory(train, image_size=(256, 256), labels="inferred", label_mode="binary", validation_split=0.2, subset="training",seed=42)
train_data = train_data.map(scale_images)
train_size = int(len(train_data))
train_data_final = train_data.take(train_size)
print(train_size)

Found 8863 files belonging to 2 classes.
Using 7091 files for training.
222


In [6]:
val_data = tf.keras.utils.image_dataset_from_directory(train, image_size=(256, 256), labels="inferred", label_mode="binary", validation_split=0.2, subset="validation", seed=42)
val_data = val_data.map(scale_images)
val_data_final = val_data.take(len(val_data))
print(len(val_data))

Found 8863 files belonging to 2 classes.
Using 1772 files for validation.
56


In [7]:
#The Model

In [9]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, AveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from hyperopt import hp, fmin, tpe, Trials

# Define the search space for hyperparameters
space = {
    'learning_rate': hp.choice('learning_rate', [1e-3, 1e-4, 1e-5, 1e-6]),  # Learning rate choice
    'optimizer': hp.choice('optimizer', [Adam, SGD, RMSprop]),   # Optimizer choice
    'neurons': hp.choice('neurons', [64, 128, 256]),    # Number of neurons in the activation layer
    'activation_function': hp.choice('activation_function', ['relu', 'tanh', 'sigmoid'])  # Activation function choice
}

# Define the objective function to optimize
def objective(params):
    model = Sequential()

    model.add(Conv2D(16, (3, 3), activation=params['activation_function'], input_shape=(256, 256, 3)))
    model.add(AveragePooling2D())

    model.add(Conv2D(32, (3, 3), activation=params['activation_function']))
    model.add(MaxPooling2D())

    model.add(Conv2D(16, (3, 3), activation=params['activation_function']))
    model.add(MaxPooling2D())

    model.add(Conv2D(32, (3, 3), activation=params['activation_function']))
    model.add(MaxPooling2D())

    model.add(Conv2D(16, (3, 3), activation=params['activation_function']))
    model.add(MaxPooling2D())

    model.add(Flatten())

    model.add(Dense(params['neurons'], activation=params['activation_function']))

    model.add(Dense(1, activation='sigmoid'))

    optimizer = params['optimizer'](learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
    history = model.fit(train_data_final, epochs=10, validation_data=val_data_final, callbacks=[early_stopping],verbose=0)
    
    # Calculate the validation accuracy
    val_accuracy = np.max(history.history['val_accuracy'])
    return -val_accuracy  # We negate it because hyperopt minimizes the objective function

# Create a Trials object to keep track of the optimization process
trials = Trials()

# Perform Bayesian optimization to find the best hyperparameters
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials)

# Print the best hyperparameters found
print("Best hyperparameters:")
print(best)

100%|████████| 20/20 [40:44<00:00, 122.23s/trial, best loss: -0.993228018283844]
Best hyperparameters:
{'activation_function': 0, 'learning_rate': 0, 'neurons': 0, 'optimizer': 2}
